In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0000_cpu_stationary_1d"
config.target = "jmlr"
config.debug_mode = True

In [ ]:
%matplotlib inline

plt.rcParams.update(config.tueplots_bundle(rel_width=0.5))

class BeliefPlotter:
    def __init__(self, bvp: linpde_gp.problems.pde.PoissonEquationDirichletProblem):
        self._bvp = bvp
        self._plt_grid = np.linspace(*bvp.domain, 100)

    def plot_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        X_pde: np.ndarray | None = None,
        f_X_pde: pn.randvars.RandomVariable | None = None,
    ):
        u_conditional_strs = []
        
        for key in conditioned_on:
            if key == "bc":
                u_conditional_strs.append(r"u\vert_{\partial \Omega} = g")
            elif key == "pde":
                u_conditional_strs.append(r"-\Delta u(x_i) = f(x_i)")
        
        u_label = (
            fr"$u \mid {', '.join(u_conditional_strs)}$"
            if len(u_conditional_strs) > 0
            else "$u$"
        )
        
        u.plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        if self._bvp.solution is not None:
            ax.plot(
                self._plt_grid,
                self._bvp.solution(self._plt_grid),
                color="C1",
                label="$u^*$",
            )
        
        for key in conditioned_on:
            if key == "bc":
                g = self._bvp.boundary_conditions[0].values

                ax.errorbar(
                    self._bvp.domain.boundary,
                    g.mean,
                    yerr=1.96 * g.std,
                    fmt="+",
                    capsize=2,
                    color="C2",
                    label=r"$g$",
                )
            elif key == "pde":
                linpde_gp.utils.plotting.plot_local_curvature(
                    ax,
                    xs=X_pde,
                    f_xs=u.mean(X_pde),
                    ddf_xs=-f_X_pde,
                    # df_xs=(
                    #     linpde_gp.linfuncops.diffops.PartialDerivative(
                    #         domain_shape=(),
                    #         domain_index=(),
                    #     )(u)(X_pde).mean
                    # ),
                    color="C3",
                    label=f"$(f(x_1), \dots, f(x_{{{X_pde.shape[0]}}}))$",
                )
        
        ax.legend()

    def plot_pred_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        X_pde: np.ndarray | None = None,
        f_X_pde: pn.randvars.RandomVariable | None = None,
    ):
        u_conditional_strs = []
        
        for key in conditioned_on:
            if key == "bc":
                u_conditional_strs.append(r"u\vert_{\partial \Omega} = g")
            elif key == "pde":
                u_conditional_strs.append(r"-\Delta u(x_i) = f(x_i)")
        
        u_label = (
            fr"$-\Delta u \mid {', '.join(u_conditional_strs)}$"
            if len(u_conditional_strs) > 0
            else "$-\Delta u$"
        )

        self._bvp.diffop(u).plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        self._bvp.rhs.plot(
            ax,
            self._plt_grid,
            color="C1",
            label="$f$",
        )
        
        if "pde" in conditioned_on:
            ax.errorbar(
                X_pde,
                f_X_pde.mean,
                yerr=1.96 * f_X_pde.std,
                fmt="+",
                capsize=2,
                c="C3",
                label=f"$(f(x_1), \dots, f(x_{{{X_pde.shape[0]}}}))$",
            )
        
        ax.legend()

## Problem Definition

In [ ]:
# https://en.wikichip.org/wiki/intel/microarchitectures/coffee_lake#Quad-Core
# https://ark.intel.com/content/www/us/en/ark/products/134896/intel-core-i59600k-processor-9m-cache-up-to-4-60-ghz.html
w_CPU = 9.19  # mm
l_CPU = 16.28  # mm
area_CPU = w_CPU * l_CPU  # mm^2

TDP_CPU = 95 # W

rel_core_positions = np.array([0.2, 0.4, 0.6])
std_core = 1.2

In [ ]:
domain = linpde_gp.domains.asdomain([0, l_CPU])

In [ ]:
# Heat Sources
f_CPU = pn.LambdaFunction(
    lambda x: (TDP_CPU / w_CPU) * (1 / np.sqrt(2 * np.pi * std_core)) * np.mean(np.exp(-(1 / (2 * std_core ** 2)) * (x[..., None] - l_CPU * rel_core_positions) ** 2), axis=-1),
    input_shape=(),
)

# Heat Sink
f_cooler = pn.LambdaFunction(
    lambda x: np.full_like(x, TDP_CPU / (area_CPU)),
    input_shape=(),
)

f = pn.LambdaFunction(
    lambda x: f_CPU(x) - f_cooler(x),
    input_shape=(),
)

In [ ]:
g = np.zeros(2) + 50

X_pde = np.linspace(0.2, 14.8, 15)

In [ ]:
bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(
    domain=domain,
    rhs=linpde_gp.randprocs.DeterministicProcess(f),
    boundary_values=pn.randvars.asrandvar(g),
    solution=None,
)

f_X_pde = bvp.rhs(X_pde)

plotter = BeliefPlotter(bvp)

In [ ]:
xs = np.linspace(0, 16.28, 100)
plt.plot(xs, f(xs))

# Prior

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=50.0),
    cov=10.0 ** 2 * linpde_gp.randprocs.kernels.Matern(
        input_shape=(),
        p=3,
        lengthscale=5.0,
    ),
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_prior,
)

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_prior,
)

## Posterior (PDE First)

### Conditioning on the PDE

In [ ]:
u_cond_pde = u_prior.condition_on_observations(
    X=X_pde,
    Y=np.zeros_like(X_pde),
    L=bvp.diffop,
    b=-f_X_pde,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

# experiment_utils.savefig("pdefirst_01a_u_cond_pde")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

# experiment_utils.savefig("pdefirst_01b_Du_cond_pde")

In [ ]:
u_cond_pde_bc = u_cond_pde.condition_on_observations(
    X=bvp.domain.boundary,
    Y=np.zeros_like(bvp.domain.boundary),
    b=-bvp.boundary_conditions[0].values,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde", "bc"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("pdefirst_02a_u_cond_pde_bc")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("pdefirst_02b_Du_cond_pde_bc")

In [ ]:
u_cond_pde_meas = u_cond_pde.condition_on_observations(
    X=rel_core_positions * l_CPU + 0.2,
    Y=np.full_like(rel_core_positions, 60),
    b=0.5 * pn.randvars.Normal(
        np.zeros_like(rel_core_positions),
        np.eye(rel_core_positions.size),
    ),
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_meas,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

# experiment_utils.savefig("pdefirst_02a_u_cond_pde_bc")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_meas,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

# experiment_utils.savefig("pdefirst_02b_Du_cond_pde_bc")